In [ ]:
#Importing required packages
#Make sure you have installed all the packages below
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
rc('text', usetex=True)
from tqdm import tqdm
import copy

from OscProbs_Python import GetProb3x3

In [ ]:
L_NOvA, E_NOvA, rho_NOvA = 810.0, 1.9, 2.84
L_T2K, E_T2K, rho_T2K = 295.0, 0.6, 2.6

In [ ]:
#best-fit oscillation parameters from www.nu-fit.org
#v5.2 (Nov. 2022)

#normal ordering, with Super-Kamiokande atmospheric data
NO_bfp = {"sinsq_t12":0.303,
          "sinsq_t13":0.02225,
          "sinsq_t23":0.451,
          "delta":232*np.pi/180,
          "dm_21":7.41e-05,
          "dm_31":2.507e-03}

#inverted ordering, with Super-Kamiokande atmospheric data
IO_bfp = {"sinsq_t12":0.303,
          "sinsq_t13":0.02223,
          "sinsq_t23":0.569,
          "delta":276*np.pi/180,
          "dm_21":7.41e-05,
          "dm_31":-2.486e-03+7.41e-05}

In [ ]:
delta_vector = np.linspace(0, 2*np.pi, 100)
NO_oscillation_probabilities = copy.deepcopy(NO_bfp)
IO_oscillation_probabilities = copy.deepcopy(IO_bfp)
p_numu_nue_NOvA_NO, p_numu_nue_NOvA_IO = [], []
for delta_test in delta_vector:
    NO_oscillation_probabilities['delta'] = delta_test
    pmue = GetProb3x3([E_NOvA], 1, NO_oscillation_probabilities, L_NOvA, rho_NOvA)[0][0][1]
    pmuebar = GetProb3x3([E_NOvA], -1, NO_oscillation_probabilities, L_NOvA, rho_NOvA)[0][0][1]
    p_numu_nue_NOvA_NO.append([pmue, pmuebar])

    IO_oscillation_probabilities['delta'] = delta_test
    pmue = GetProb3x3([E_NOvA], 1, IO_oscillation_probabilities, L_NOvA, rho_NOvA)[0][0][1]
    pmuebar = GetProb3x3([E_NOvA], -1, IO_oscillation_probabilities, L_NOvA, rho_NOvA)[0][0][1]
    p_numu_nue_NOvA_IO.append([pmue, pmuebar])

In [ ]:
figwid = 4.0
fighei = 4.0
lside = 1.1
rside = 0.1
wwspace = 1.5

ncol = 1
nrow = 1

wid = lside + ncol*figwid + (ncol-1)*wwspace + rside

bot = 0.9
top = 0.1
hhspace = 1.25

hei = bot + nrow*fighei + (nrow-1)*hhspace + top

lfactor = lside/wid
rfactor = rside/wid
bfactor = bot/hei
tfactor = top/hei
wfactor = wwspace/figwid
hfactor = hhspace/fighei

fig, axes = plt.subplots(nrow, ncol, figsize=(wid, hei), facecolor='1.0');
fig.subplots_adjust(left = lfactor, bottom=bfactor, right=(1.0-rfactor), top=(1.0-tfactor), wspace=wfactor, hspace=hfactor);

ax = axes
xmin = 0
xmax = 0.1
ymin = 0
ymax = 0.1
ax.axis([xmin, xmax, ymin, ymax])
ax.set_xlabel(r'$P(\nu_\mu \to \nu_e)$', fontsize=16)        
ax.set_ylabel(r'$P(\bar\nu_\mu \to \bar\nu_e)$', fontsize=16)        
            
ax.tick_params(direction='in', reset=True, which='both', zorder=30)
[l.set_position((0.5, -0.015)) for l in ax.get_xticklabels()]
[l.set_size(12) for l in ax.get_xticklabels()]
[l.set_size(12) for l in ax.get_yticklabels()]

ax.set_xlim([xmin,xmax])
ax.set_ylim([ymin,ymax])

ax.plot(np.transpose(p_numu_nue_NOvA_NO)[0], np.transpose(p_numu_nue_NOvA_NO)[1], color='dodgerblue', ls='-', lw=2, zorder=10)
ax.plot(np.transpose(p_numu_nue_NOvA_IO)[0], np.transpose(p_numu_nue_NOvA_IO)[1], color='dodgerblue', ls='--', lw=2, zorder=10)
ax.plot([0, 1], [0, 1], 'k--', lw=1)

In [ ]:
#Denton et al [2008.01110]
nova_nue_xyz_Denton = [1149.7, 0.0, 31.15]
nova_nuebar_xyz_Denton = [22.96, 472.60, 13.97]

t2k_nue_xyz_Denton = [1284.16, 36.90, 19.71]
t2k_nuebar_xyz_Denton = [49.51, 231.32, 5.84]

#Ellis et al [2008.01088]
nova_nue_xyz = [1202.65, 0.0, 29.09]
nova_nuebar_xyz = [0.0, 438.426, 16.59]

t2k_nue_xyz = [1284.84, 0, 21.90]
t2k_nuebar_xyz = [0, 179.59, 10.66]

def app_events(osc_params, L=L_NOvA, E=E_NOvA, rho=rho_NOvA, xyz_nue=nova_nue_xyz, xyz_nuebar=nova_nuebar_xyz):
    prob_nue = GetProb3x3([E], 1, osc_params, L, rho)[0][0][1]
    prob_nuebar = GetProb3x3([E], -1, osc_params, L, rho)[0][0][1]

    x, y, z = xyz_nue
    nu_e_evts = x*prob_nue + y*prob_nuebar + z
    x, y, z = xyz_nuebar
    nu_e_bar_evts = x*prob_nue + y*prob_nuebar + z

    return [nu_e_evts, nu_e_bar_evts]

nova_evts_true = [82, 33]
t2k_evts_true = [108, 16]
def chi2_app(osc_params, L=L_NOvA, E=E_NOvA, rho=rho_NOvA, xyz_nue=nova_nue_xyz, xyz_nuebar=nova_nuebar_xyz, data=nova_evts_true):
    evts_test = app_events(osc_params, L, E, rho, xyz_nue, xyz_nuebar)

    lnltest = 0.0
    l = evts_test[0]
    x = data[0]
    lnltest += -l + x + x*np.log(l/x)

    l = evts_test[1]
    x = data[1]
    lnltest += -l + x + x*np.log(l/x)

    return -2.0*lnltest

dm32_NOvA = [2.41e-03, 0.07e-03] #central value/error bar for NOvA \Delta m_{32}^2 measurement
dm32_T2K = [2.49e-03, 0.082e-03] #central value/error bar for T2K \Delta m_{32}^2 measurement

C_NOvA = [0.99, 0.02] #NOvA measurement of 4|U_{\mu3}^2|(1-|U_{\mu3}|^2)
C_T2K = [1.00, 0.03] #T2K measurement of 4|U_{\mu3}^2|(1-|U_{\mu3}|^2)

def chi2_dis(osc_params, dm32_info=dm32_NOvA, C_info=C_NOvA):
    dm32 = osc_params['dm_31'] - osc_params['dm_21']
    dm32_true, dm32_err = dm32_info
    dm32_chi2 = (np.abs(dm32) - dm32_true)**2/(dm32_err**2)

    Um3sq = osc_params['sinsq_t23']*(1-osc_params['sinsq_t13'])
    C = 4*Um3sq*(1-Um3sq)
    C_true, C_err = C_info
    C_chi2 = (C - C_true)**2/(C_err**2)

    return dm32_chi2 + C_chi2

In [ ]:
delta_vector = np.linspace(0, 2*np.pi, 100)
NO_oscillation_probabilities = copy.deepcopy(NO_bfp)
IO_oscillation_probabilities = copy.deepcopy(IO_bfp)

evts_NOvA_NO, evts_NOvA_IO = [], []

evts_T2K_NO, evts_T2K_IO = [], []

for delta_test in delta_vector:
    NO_oscillation_probabilities['delta'] = delta_test
    nue_evts = app_events(NO_oscillation_probabilities)
    evts_NOvA_NO.append(nue_evts)

    IO_oscillation_probabilities['delta'] = delta_test
    nue_evts = app_events(IO_oscillation_probabilities)
    evts_NOvA_IO.append(nue_evts)

    nue_evts = app_events(NO_oscillation_probabilities, L=L_T2K, E=E_T2K, xyz_nue=t2k_nue_xyz, xyz_nuebar=t2k_nuebar_xyz)
    evts_T2K_NO.append(nue_evts)

    IO_oscillation_probabilities['delta'] = delta_test
    nue_evts = app_events(IO_oscillation_probabilities, L=L_T2K, E=E_T2K, xyz_nue=t2k_nue_xyz, xyz_nuebar=t2k_nuebar_xyz)
    evts_T2K_IO.append(nue_evts)

In [ ]:
figwid = 4.0
fighei = 4.0
lside = 1.1
rside = 0.1
wwspace = 1.5

ncol = 2
nrow = 1

wid = lside + ncol*figwid + (ncol-1)*wwspace + rside

bot = 0.9
top = 0.1
hhspace = 1.25

hei = bot + nrow*fighei + (nrow-1)*hhspace + top

lfactor = lside/wid
rfactor = rside/wid
bfactor = bot/hei
tfactor = top/hei
wfactor = wwspace/figwid
hfactor = hhspace/fighei

fig, axes = plt.subplots(nrow, ncol, figsize=(wid, hei), facecolor='1.0');
fig.subplots_adjust(left = lfactor, bottom=bfactor, right=(1.0-rfactor), top=(1.0-tfactor), wspace=wfactor, hspace=hfactor);

ax = axes[0]
xmin = 15
xmax = 130
ymin = 12
ymax = 62
ax.axis([xmin, xmax, ymin, ymax])
ax.set_xlabel(r'$N_{\nu_e}$', fontsize=16)        
ax.set_ylabel(r'$N_{\bar\nu_e}$', fontsize=16)        
            
ax.tick_params(direction='in', reset=True, which='both', zorder=30)
[l.set_position((0.5, -0.015)) for l in ax.get_xticklabels()]
[l.set_size(12) for l in ax.get_xticklabels()]
[l.set_size(12) for l in ax.get_yticklabels()]

ax.set_xlim([xmin,xmax])
ax.set_ylim([ymin,ymax])

ax.plot(np.transpose(evts_NOvA_NO)[0], np.transpose(evts_NOvA_NO)[1], color='dodgerblue', ls='-', lw=2, zorder=10)
ax.plot(np.transpose(evts_NOvA_IO)[0], np.transpose(evts_NOvA_IO)[1], color='firebrick', ls='--', lw=2, zorder=10)

ax.errorbar(nova_evts_true[0], nova_evts_true[1], xerr=np.sqrt(nova_evts_true[0]), yerr=np.sqrt(nova_evts_true[1]), color='k', zorder=0)

ax = axes[1]
xmin = 40
xmax = 130
ymin = 8
ymax = 25
ax.axis([xmin, xmax, ymin, ymax])
ax.set_xlabel(r'$N_{\nu_e}$', fontsize=16)        
ax.set_ylabel(r'$N_{\bar\nu_e}$', fontsize=16)        
            
ax.tick_params(direction='in', reset=True, which='both', zorder=30)
[l.set_position((0.5, -0.015)) for l in ax.get_xticklabels()]
[l.set_size(12) for l in ax.get_xticklabels()]
[l.set_size(12) for l in ax.get_yticklabels()]

ax.set_xlim([xmin,xmax])
ax.set_ylim([ymin,ymax])

ax.plot(np.transpose(evts_T2K_NO)[0], np.transpose(evts_T2K_NO)[1], color='dodgerblue', ls='-', lw=2, zorder=10)
ax.plot(np.transpose(evts_T2K_IO)[0], np.transpose(evts_T2K_IO)[1], color='firebrick', ls='--', lw=2, zorder=10)
ax.errorbar(t2k_evts_true[0], t2k_evts_true[1], xerr=np.sqrt(t2k_evts_true[0]), yerr=np.sqrt(t2k_evts_true[1]), color='k', zorder=0)


In [ ]:
delta_vector = np.linspace(-np.pi, np.pi, 100)
NO_oscillation_probabilities = {"sinsq_t12":0.304,
                                "sinsq_t13":0.0219,
                                "sinsq_t23":0.55,
                                "dm_21":7.53e-05,
                                "dm_31":2.565e-03}
IO_oscillation_probabilities = {"sinsq_t12":0.304,
                                "sinsq_t13":0.0219,
                                "sinsq_t23":0.55,
                                "dm_21":7.53e-05,
                                "dm_31":-2.46e-03}

evts_T2K_NO_v2, evts_T2K_IO_v2 = [], []
evts_T2K_NO_v3, evts_T2K_IO_v3 = [], []

for delta_test in delta_vector:
    NO_oscillation_probabilities['delta'] = delta_test
    nue_evts = app_events(NO_oscillation_probabilities, L=L_T2K, E=E_T2K, xyz_nue=t2k_nue_xyz, xyz_nuebar=t2k_nuebar_xyz)
    evts_T2K_NO_v2.append(nue_evts)
    nue_evts = app_events(NO_oscillation_probabilities, L=L_T2K, E=E_T2K, xyz_nue=t2k_nue_xyz_Denton, xyz_nuebar=t2k_nuebar_xyz_Denton)
    evts_T2K_NO_v3.append(nue_evts)

    IO_oscillation_probabilities['delta'] = delta_test
    nue_evts = app_events(IO_oscillation_probabilities, L=L_T2K, E=E_T2K, xyz_nue=t2k_nue_xyz, xyz_nuebar=t2k_nuebar_xyz)
    evts_T2K_IO_v2.append(nue_evts)
    nue_evts = app_events(IO_oscillation_probabilities, L=L_T2K, E=E_T2K, xyz_nue=t2k_nue_xyz_Denton, xyz_nuebar=t2k_nuebar_xyz_Denton)
    evts_T2K_IO_v3.append(nue_evts)

In [ ]:
#T2K Validation
delta_test = [-np.pi, -np.pi/2, 0, np.pi/2, np.pi]
NO_nu_T2KPlot = [88.7, 105.27, 88.9, 72.4, 88.7]
NO_nubar_T2KPlot = [18.3, 16.4, 18.7, 20.6, 18.3]

IO_nu_T2KPlot = [78.5, 93.2, 77.9, 63.2, 78.5]
IO_nubar_T2KPlot = [20.1, 17.6, 19.9, 22.4, 20.1]

In [ ]:
figwid = 4.0
fighei = 4.0
lside = 1.1
rside = 0.1
wwspace = 0.6

ncol = 2
nrow = 1

wid = lside + ncol*figwid + (ncol-1)*wwspace + rside

bot = 0.9
top = 0.1
hhspace = 1.25

hei = bot + nrow*fighei + (nrow-1)*hhspace + top

lfactor = lside/wid
rfactor = rside/wid
bfactor = bot/hei
tfactor = top/hei
wfactor = wwspace/figwid
hfactor = hhspace/fighei

fig, axes = plt.subplots(nrow, ncol, figsize=(wid, hei), facecolor='1.0');
fig.subplots_adjust(left = lfactor, bottom=bfactor, right=(1.0-rfactor), top=(1.0-tfactor), wspace=wfactor, hspace=hfactor);

ax = axes[0]
xmin = -np.pi
xmax = np.pi
ymin = 0
ymax = 150
ax.axis([xmin, xmax, ymin, ymax])
ax.set_xlabel(r'$\delta_{\rm CP}$', fontsize=16)        
ax.set_ylabel(r'$N_{\nu_e}$', fontsize=16)        
            
ax.tick_params(direction='in', reset=True, which='both', zorder=30)
[l.set_position((0.5, -0.015)) for l in ax.get_xticklabels()]
[l.set_size(12) for l in ax.get_xticklabels()]
[l.set_size(12) for l in ax.get_yticklabels()]

ax.set_xlim([xmin,xmax])
ax.set_ylim([ymin,ymax])

ax.plot(delta_vector, np.transpose(evts_T2K_NO_v2)[0], color='dodgerblue', ls='-', lw=2, zorder=10)
ax.plot(delta_vector, np.transpose(evts_T2K_NO_v3)[0], color='dodgerblue', ls='--', lw=2, zorder=10)
ax.plot(delta_vector, np.transpose(evts_T2K_IO_v2)[0], color='firebrick', ls='-', lw=2, zorder=10)
ax.plot(delta_vector, np.transpose(evts_T2K_IO_v3)[0], color='firebrick', ls='--', lw=2, zorder=10)

ax.plot(delta_test, NO_nu_T2KPlot, color='dodgerblue', lw=0, marker='*', ms=8, zorder=20)
ax.plot(delta_test, IO_nu_T2KPlot, color='firebrick', lw=0, marker='*', ms=8, zorder=20)

ax = axes[1]
xmin = -np.pi
xmax = np.pi
ymin = 0
ymax = 30
ax.axis([xmin, xmax, ymin, ymax])
ax.set_xlabel(r'$\delta_{\rm CP}$', fontsize=16)        
ax.set_ylabel(r'$N_{\bar\nu_e}$', fontsize=16)        
            
ax.tick_params(direction='in', reset=True, which='both', zorder=30)
[l.set_position((0.5, -0.015)) for l in ax.get_xticklabels()]
[l.set_size(12) for l in ax.get_xticklabels()]
[l.set_size(12) for l in ax.get_yticklabels()]

ax.set_xlim([xmin,xmax])
ax.set_ylim([ymin,ymax])

ax.plot(delta_vector, np.transpose(evts_T2K_NO_v2)[1], color='dodgerblue', ls='-', lw=2, zorder=10)
ax.plot(delta_vector, np.transpose(evts_T2K_NO_v3)[1], color='dodgerblue', ls='--', lw=2, zorder=10)
ax.plot(delta_vector, np.transpose(evts_T2K_IO_v2)[1], color='firebrick', ls='-', lw=2, zorder=10)
ax.plot(delta_vector, np.transpose(evts_T2K_IO_v3)[1], color='firebrick', ls='--', lw=2, zorder=10)

ax.plot(delta_test, NO_nubar_T2KPlot, color='dodgerblue', lw=0, marker='*', ms=8, zorder=20)
ax.plot(delta_test, IO_nubar_T2KPlot, color='firebrick', lw=0, marker='*', ms=8, zorder=20)


In [ ]:
dm31_vec = np.linspace(2.0e-3, 3.0e-3, 50)
ssq23_vec = np.linspace(0.3, 0.7, 50)
params = copy.deepcopy(NO_bfp)

t2k_chi2 = np.zeros((len(dm31_vec), len(ssq23_vec)))
nova_chi2 = np.zeros((len(dm31_vec), len(ssq23_vec)))

for dm31i, dm31 in tqdm(enumerate(dm31_vec)):
    params['dm_31'] = dm31
    for ssq23i, ssq23 in enumerate(ssq23_vec):
        params['sinsq_t23'] = ssq23

        t2k_chi2_dis = chi2_dis(params, dm32_info=dm32_T2K, C_info=C_T2K)
        nova_chi2_dis = chi2_dis(params, dm32_info=dm32_NOvA, C_info=C_NOvA)

        t2k_chi2[dm31i, ssq23i] = t2k_chi2_dis
        nova_chi2[dm31i, ssq23i] = nova_chi2_dis

In [ ]:
figwid = 4.0
fighei = 4.0
lside = 1.1
rside = 0.1
wwspace = 1.5

ncol = 1
nrow = 1

wid = lside + ncol*figwid + (ncol-1)*wwspace + rside

bot = 0.9
top = 0.1
hhspace = 1.25

hei = bot + nrow*fighei + (nrow-1)*hhspace + top

lfactor = lside/wid
rfactor = rside/wid
bfactor = bot/hei
tfactor = top/hei
wfactor = wwspace/figwid
hfactor = hhspace/fighei

fig, axes = plt.subplots(nrow, ncol, figsize=(wid, hei), facecolor='1.0');
fig.subplots_adjust(left = lfactor, bottom=bfactor, right=(1.0-rfactor), top=(1.0-tfactor), wspace=wfactor, hspace=hfactor);

ax = axes
xmin = 0.3
xmax = 0.7
ymin = 2.0e-3
ymax = 3.0e-3
ax.axis([xmin, xmax, ymin, ymax])
ax.set_xlabel(r'$\sin^2\theta_{23}$', fontsize=16)        
ax.set_ylabel(r'$\Delta m_{31}^2\ [\mathrm{eV}^2]$', fontsize=16)        
            
ax.tick_params(direction='in', reset=True, which='both', zorder=30)
[l.set_position((0.5, -0.015)) for l in ax.get_xticklabels()]
[l.set_size(12) for l in ax.get_xticklabels()]
[l.set_size(12) for l in ax.get_yticklabels()]

ax.set_xlim([xmin,xmax])
ax.set_ylim([ymin,ymax])

ax.contour(ssq23_vec, dm31_vec, t2k_chi2, levels=[5.99], colors='mediumseagreen')
ax.contour(ssq23_vec, dm31_vec, nova_chi2, levels=[5.99], colors='mediumpurple', linestyles='--')

In [ ]:
dm31_vec = np.linspace(2.0e-3, 3.0e-3, 50)
ssq23_vec = np.linspace(0.3, 0.7, 50)
dcp_vec = np.linspace(-np.pi, np.pi, 50)

params = copy.deepcopy(NO_bfp)

t2k_chi2 = np.zeros((len(dm31_vec), len(ssq23_vec), len(dcp_vec)))
nova_chi2 = np.zeros((len(dm31_vec), len(ssq23_vec), len(dcp_vec)))

for dm31i, dm31 in tqdm(enumerate(dm31_vec)):
    params['dm_31'] = dm31
    for ssq23i, ssq23 in enumerate(ssq23_vec):
        params['sinsq_t23'] = ssq23

        for dcpi, dcp in enumerate(dcp_vec):
            params['delta'] = dcp

            t2k_chi2_dis = chi2_dis(params, dm32_info=dm32_T2K, C_info=C_T2K)
            nova_chi2_dis = chi2_dis(params, dm32_info=dm32_NOvA, C_info=C_NOvA)

            t2k_chi2_app = chi2_app(params, L=L_T2K, E=E_T2K, rho=rho_T2K, xyz_nue=t2k_nue_xyz, xyz_nuebar=t2k_nuebar_xyz, data=t2k_evts_true)
            nova_chi2_app = chi2_app(params, L=L_NOvA, E=E_NOvA, rho=rho_NOvA, xyz_nue=nova_nue_xyz, xyz_nuebar=nova_nuebar_xyz, data=nova_evts_true)

            t2k_chi2[dm31i, ssq23i, dcpi] = t2k_chi2_dis + t2k_chi2_app
            nova_chi2[dm31i, ssq23i, dcpi] = nova_chi2_dis + nova_chi2_app

In [ ]:
dm31_vec_IO = np.linspace(-3.0e-3, -2.0e-3, 50)
ssq23_vec = np.linspace(0.3, 0.7, 50)
dcp_vec = np.linspace(-np.pi, np.pi, 50)

params = copy.deepcopy(IO_bfp)

t2k_chi2_IO = np.zeros((len(dm31_vec), len(ssq23_vec), len(dcp_vec)))
nova_chi2_IO = np.zeros((len(dm31_vec), len(ssq23_vec), len(dcp_vec)))

for dm31i, dm31 in tqdm(enumerate(dm31_vec_IO)):
    params['dm_31'] = dm31
    for ssq23i, ssq23 in enumerate(ssq23_vec):
        params['sinsq_t23'] = ssq23

        for dcpi, dcp in enumerate(dcp_vec):
            params['delta'] = dcp

            t2k_chi2_dis = chi2_dis(params, dm32_info=dm32_T2K, C_info=C_T2K)
            nova_chi2_dis = chi2_dis(params, dm32_info=dm32_NOvA, C_info=C_NOvA)

            t2k_chi2_app = chi2_app(params, L=L_T2K, E=E_T2K, rho=rho_T2K, xyz_nue=t2k_nue_xyz, xyz_nuebar=t2k_nuebar_xyz, data=t2k_evts_true)
            nova_chi2_app = chi2_app(params, L=L_NOvA, E=E_NOvA, rho=rho_NOvA, xyz_nue=nova_nue_xyz, xyz_nuebar=nova_nuebar_xyz, data=nova_evts_true)

            t2k_chi2_IO[dm31i, ssq23i, dcpi] = t2k_chi2_dis + t2k_chi2_app
            nova_chi2_IO[dm31i, ssq23i, dcpi] = nova_chi2_dis + nova_chi2_app


In [ ]:
joint_chi2 = t2k_chi2 + nova_chi2
joint_chi2_IO = t2k_chi2_IO + nova_chi2_IO

In [ ]:
nova_bf_chi2 = np.min([np.min(nova_chi2), np.min(nova_chi2_IO)])
nova_chi2_margdm31 = np.amin(nova_chi2-nova_bf_chi2, axis=0)
nova_chi2_margdm31_IO = np.amin(nova_chi2_IO-nova_bf_chi2, axis=0)

t2k_bf_chi2 = np.min([np.min(t2k_chi2), np.min(t2k_chi2_IO)])
t2k_chi2_margdm31 = np.amin(t2k_chi2-t2k_bf_chi2, axis=0)
t2k_chi2_margdm31_IO = np.amin(t2k_chi2_IO-t2k_bf_chi2, axis=0)

joint_bf_chi2 = np.min([np.min(joint_chi2), np.min(joint_chi2_IO)])
joint_chi2_margdm31 = np.amin(joint_chi2 - joint_bf_chi2, axis=0)
joint_chi2_margdm31_IO = np.amin(joint_chi2_IO - joint_bf_chi2, axis=0)

In [ ]:
print(np.min(nova_chi2_margdm31), np.min(t2k_chi2_margdm31))
print(np.min(nova_chi2_margdm31_IO), np.min(t2k_chi2_margdm31_IO))
print(np.min(joint_chi2_margdm31), np.min(joint_chi2_margdm31_IO))

In [ ]:
nova_chi2_dcp = np.min(nova_chi2_margdm31, axis=0)
t2k_chi2_dcp = np.min(t2k_chi2_margdm31, axis=0)

nova_chi2_dcp_IO = np.min(nova_chi2_margdm31_IO, axis=0)
t2k_chi2_dcp_IO = np.min(t2k_chi2_margdm31_IO, axis=0)

joint_chi2_dcp = np.min(joint_chi2_margdm31, axis=0)
joint_chi2_dcp_IO = np.min(joint_chi2_margdm31_IO, axis=0)

In [ ]:
figwid = 4.0
fighei = 2.0
lside = 1.1
rside = 0.1
wwspace = 1.5

ncol = 1
nrow = 3

wid = lside + ncol*figwid + (ncol-1)*wwspace + rside

bot = 0.9
top = 0.1
hhspace = 0.15

hei = bot + nrow*fighei + (nrow-1)*hhspace + top

lfactor = lside/wid
rfactor = rside/wid
bfactor = bot/hei
tfactor = top/hei
wfactor = wwspace/figwid
hfactor = hhspace/fighei

fig, axes = plt.subplots(nrow, ncol, figsize=(wid, hei), facecolor='1.0');
fig.subplots_adjust(left = lfactor, bottom=bfactor, right=(1.0-rfactor), top=(1.0-tfactor), wspace=wfactor, hspace=hfactor);

ax = axes[1]
xmin = -np.pi
xmax = np.pi
ymin = 0.3
ymax = 0.7
ax.axis([xmin, xmax, ymin, ymax])
ax.set_ylabel(r'$\sin^2\theta_{23}$', fontsize=16)        
ax.set_xlabel(r'', fontsize=16)        
            
ax.tick_params(direction='in', reset=True, which='both', zorder=30)
[l.set_position((0.5, -0.015)) for l in ax.get_xticklabels()]
[l.set_size(0) for l in ax.get_xticklabels()]
[l.set_size(12) for l in ax.get_yticklabels()]

ax.set_xlim([xmin,xmax])
ax.set_ylim([ymin,ymax])

ax.contour(dcp_vec, ssq23_vec, t2k_chi2_margdm31, levels=[2.3, 4.61], colors='coral', linestyles=['--', '-'])
ax.contour(dcp_vec, ssq23_vec, nova_chi2_margdm31, levels=[2.3, 4.61], colors='dodgerblue', linestyles=['--', '-'])
ax.contour(dcp_vec, ssq23_vec, joint_chi2_margdm31, levels=[2.3, 4.61], colors='k', linestyles=['--', '-'])

ax = axes[2]
xmin = -np.pi
xmax = np.pi
ymin = 0.3
ymax = 0.7
ax.axis([xmin, xmax, ymin, ymax])
ax.set_ylabel(r'$\sin^2\theta_{23}$', fontsize=16)        
ax.set_xlabel(r'$\delta_{\rm CP}$', fontsize=16)        
            
ax.tick_params(direction='in', reset=True, which='both', zorder=30)
[l.set_position((0.5, -0.015)) for l in ax.get_xticklabels()]
[l.set_size(12) for l in ax.get_xticklabels()]
[l.set_size(12) for l in ax.get_yticklabels()]

ax.set_xlim([xmin,xmax])
ax.set_ylim([ymin,ymax])

ax.contour(dcp_vec, ssq23_vec, t2k_chi2_margdm31_IO, levels=[2.3, 4.61], colors='coral', linestyles=['--', '-'])
ax.contour(dcp_vec, ssq23_vec, nova_chi2_margdm31_IO, levels=[2.3, 4.61], colors='dodgerblue', linestyles=['--', '-'])
ax.contour(dcp_vec, ssq23_vec, joint_chi2_margdm31_IO, levels=[2.3, 4.61], colors='k', linestyles=['--', '-'])

ax = axes[0]
xmin = -np.pi
xmax = np.pi
ymin = 0
ymax = 16
ax.axis([xmin, xmax, ymin, ymax])
ax.set_ylabel(r'$\Delta\chi^2$', fontsize=16)        
ax.set_xlabel(r'', fontsize=16)        
            
ax.tick_params(direction='in', reset=True, which='both', zorder=30)
[l.set_position((0.5, -0.015)) for l in ax.get_xticklabels()]
[l.set_size(0) for l in ax.get_xticklabels()]
[l.set_size(12) for l in ax.get_yticklabels()]

ax.set_xlim([xmin,xmax])
ax.set_ylim([ymin,ymax])

ax.plot(dcp_vec, t2k_chi2_dcp, color='coral')
ax.plot(dcp_vec, nova_chi2_dcp, color='dodgerblue')

ax.plot(dcp_vec, t2k_chi2_dcp_IO, color='coral', ls='--')
ax.plot(dcp_vec, nova_chi2_dcp_IO, color='dodgerblue', ls='--')

ax.plot(dcp_vec, joint_chi2_dcp, color='k')
ax.plot(dcp_vec, joint_chi2_dcp_IO, color='k', ls='--')